In [97]:
##import all the required libraries
import pandas as pd
import numpy as np
import unicodedata
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from string import ascii_lowercase
from nltk.stem import WordNetLemmatizer
from collections import Counter
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
import operator
import matplotlib.pyplot as plt
from collections import Counter
from string import ascii_lowercase
import unicodedata
import nltk
import re
from nltk.stem import PorterStemmer
from gensim.models import word2vec
from sklearn import manifold
import itertools
import time
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models
from gensim import similarities
import heapq

In [98]:
df_ingre = pd.read_csv('recipe_dataset/recipe_ingredient.csv', encoding="cp949")
df_basic = pd.read_csv('recipe_dataset/recipe_basic.csv', encoding="cp949")

df_tmp=df_ingre[['RECIPE_ID','IRDNT_NM']]
df_tmp_arr=[[]for i in range(538)]
num=0
for i in df_tmp['IRDNT_NM'] :
    df_tmp_arr[df_tmp['RECIPE_ID'][num]].append(i)
    num+=1
    
df_tmp_arr.pop(0)
num=0
for i in df_tmp_arr :
    df_tmp_arr[num] = list(set(df_tmp_arr[num]))
    num+=1
    
ingre_dict = {}

for i in range(1, 538, 1):
    ingre_dict[str(i)] = df_tmp_arr[i-1]

ingre_dict

{'1': ['설탕',
  '양지머리',
  '고사리',
  '다진파',
  '계란',
  '안심',
  '청포묵',
  '쌀',
  '도라지',
  '고추장',
  '미나리',
  '콩나물',
  '숙주',
  '참기름',
  '소금',
  '다진마늘',
  '국간장'],
 '2': ['팥', '차조', '수수', '찹쌀', '멥쌀', '콩', '소금'],
 '3': ['당근',
  '돼지고기',
  '홍고추',
  '다진파',
  '청고추',
  '호박',
  '통후추',
  '밥',
  '부추',
  '진간장',
  '참기름',
  '소금',
  '다진마늘',
  '당면',
  '표고버섯'],
 '4': ['마늘', '깨소금', '파', '통후추', '쌀', '쇠고기', '진간장', '참기름', '콩나물', '고춧가루', '소금'],
 '5': ['계핏가루', '물엿', '흰설탕', '흑설탕', '대추', '찹쌀', '간장', '식용유', '깐밤', '물', '잣'],
 '6': ['팥', '청동호박', '찹쌀', '소금', '설탕', '물'],
 '7': ['흑임자', '쌀', '소금', '설탕', '물'],
 '8': ['당근', '감자', '양파', '통후추', '밥', '쇠고기', '우유', '카레가루', '소금', '완두콩'],
 '9': ['후춧가루',
  '당근',
  '멸치',
  '양파',
  '전분',
  '계란',
  '오이',
  '청피망',
  '홍피망',
  '밥',
  '소금',
  '다시마',
  '물',
  '쌈장'],
 '10': ['밀가루', '멸치', '홍고추', '감자', '실파', '양념장', '참기름', '소금', '애호박', '물', '국간장'],
 '11': ['배',
  '계란',
  '냉면',
  '오이',
  '쇠고기육수',
  '동치미무',
  '쇠고기',
  '동치미국물',
  '식초',
  '소금',
  '설탕'],
 '12': ['깨소금',
  '홍고추',
  '계란',
  '통후추',
  '동치미

In [99]:
bags_of_words = [ Counter(IRDNT_NM) for IRDNT_NM in df_tmp_arr ]

In [100]:
#Find sum of every ingredient using Counter()
sumbags = sum(bags_of_words, Counter())


# Finally, plot the 10 most used ingredients
clean_df = pd.DataFrame.from_dict(sumbags, orient='index').reset_index()
clean_df = clean_df.rename(columns={'index':'ingredient', 0:'count'})
clean_df.to_csv('ingredient_clean.csv')

top_ing = clean_df.sort_values('count', ascending=False)

In [101]:
clean_df["count"]

0      210
1        2
2        9
3      109
4       90
      ... 
639      1
640      1
641      1
642      1
643      1
Name: count, Length: 644, dtype: int64

In [102]:
ingr_only_dict = clean_df['ingredient'].to_dict()

In [103]:
#Get all clean ingredients in list format per recipe
ingr_list = []
for IRDNT_NM in df_tmp_arr:
    ingr_list.append(IRDNT_NM)

In [104]:
print (ingr_list[0])

['설탕', '양지머리', '고사리', '다진파', '계란', '안심', '청포묵', '쌀', '도라지', '고추장', '미나리', '콩나물', '숙주', '참기름', '소금', '다진마늘', '국간장']


In [105]:
##Get clean ingredients to be used as an input for word2vec model to identify ingredient similarity.

ingr_clean_df = pd.DataFrame({'Ingredient':ingr_list})
#print ingr_clean_df
ingr_clean_df

,Ingredient
0,"[설탕, 양지머리, 고사리, 다진파, 계란, 안심, 청포묵, 쌀, 도라지, 고추장,..."
1,"[팥, 차조, 수수, 찹쌀, 멥쌀, 콩, 소금]"
2,"[당근, 돼지고기, 홍고추, 다진파, 청고추, 호박, 통후추, 밥, 부추, 진간장,..."
3,"[마늘, 깨소금, 파, 통후추, 쌀, 쇠고기, 진간장, 참기름, 콩나물, 고춧가루,..."
4,"[계핏가루, 물엿, 흰설탕, 흑설탕, 대추, 찹쌀, 간장, 식용유, 깐밤, 물, 잣]"
...,...
532,"[마요네즈, 소금, 고구마, 건블루베리, 꿀, 노른자, 식빵, 우유]"
533,"[양파, 계란, 햄, 밥, 방울토마토, 샐러리, 카레가루, 후추, 물]"
534,"[통깨, 식초, 고추장, 쪽파, 중새우, 소금, 설탕]"
535,"[통깨, 당근, 마요네즈, 오이, 김, 가지, 밥, 어린잎채소, 두부, 참기름, 식..."


In [106]:
#pip install -U gensim
#Implementing word2vec to get the recipes which are similar to each other

num_features = 300   # Word vector dimensionality                      
context = 1        # Context window size; 
downsampling = 1e-3   # threshold for configuring which higher-frequency words are randomly downsampled

# Initialize and train the model 
model = word2vec.Word2Vec(ingr_list, vector_size=num_features, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

<ipython-input-106-8f38cba29e2a>:13: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [107]:
flatten_list = []
for sublist in ingr_list:
    for item in sublist:
        if item not in flatten_list:
            flatten_list.append(item)

In [108]:
most_sim_list = []
corpus_sim_dict = {}
input_list = []
for i in flatten_list:
    try: 
        if len(i) > 0:
            #print i
            corpus_sim_dict.update({i:model.wv.most_similar(i)})
    except KeyError:
        pass

In [ ]:
sim_df = pd.DataFrame([])
for key,value in corpus_sim_dict.items():
    for i in value:
        sim_df = sim_df.append(pd.DataFrame({'Ingredient': key, 'Similar Ingredient':i[0], 'Word2Vec Value': i[1] }, index=[0]), ignore_index=True)

In [ ]:
sim_df.head()

In [ ]:
from gensim.models.word2vec import Word2Vec
word2vecTSNE = Word2Vec(
    sg=1,
    vector_size=num_features, window = context, sample = downsampling,
    min_count=3
)
word2vecTSNE.build_vocab(ingr_list)

In [ ]:
word2vecTSNE.train(ingr_list, total_examples= word2vecTSNE.corpus_count, epochs=word2vecTSNE.epochs)

In [ ]:
import os
if not os.path.exists("trained"):
    os.makedirs("trained")
word2vecTSNE.save(os.path.join("trained", "word2vecTSNE.w2v"))
word2vecTSNE = word2vec.Word2Vec.load(os.path.join("trained", "word2vecTSNE.w2v"))


In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [ ]:
all_word_vectors_matrix = word2vecTSNE.wv.vectors
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

In [ ]:
#Plot the big picture

points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[word2vecTSNE.wv.key_to_index[word]])
            for word in word2vecTSNE.wv.key_to_index
        ]
    ],
    columns=["word", "x", "y"]
)


In [ ]:
sim_df.to_csv('word2vec_ingredient_similarity.csv',encoding='utf-8')

In [ ]:
df = pd.read_csv('word2vec_ingredient_similarity.csv')
df = df.sort_values('Word2Vec Value',ascending=False)
top_df = df.groupby('Ingredient').head(5)
top_df.to_csv('word2vec_ingredient_similarity_top.csv')

In [ ]:
model.wv.most_similar('마늘')

In [ ]:
#Create a dictionary for all the ingredients in the recipe list

dictionary = corpora.Dictionary(ingr_list)
#print(dictionary)
#print(dictionary.token2id)
print (dictionary.token2id)

In [ ]:
#Applying doc2bow on the dictionary of ingredients, which converts the ingredient to a number in every recipe
#This input format is needed for TfIdfmodel
bow_corpus = [dictionary.doc2bow(text) for text in ingr_list]
bow_corpus[1]

In [ ]:
# train the model
tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
#print corpus for the first recipe
for i in corpus_tfidf:
    print (i)
    break

In [ ]:
#print tfidf results for the first recipe
print(tfidf[bow_corpus[1]])

In [ ]:
#Use similarities library from gensim to get the cosine similarity of the tfidf results

index = similarities.MatrixSimilarity(tfidf[bow_corpus])
index.save('ingr.index')
index = similarities.MatrixSimilarity.load('ingr.index')

sims = index[corpus_tfidf]
sims_list = [(i,j) for i,j in enumerate(sims)]

#Creating a list to hold the cosine similarity results for tfidf
tf_idf_list = []

for i,j in enumerate(sims_list):
    tf_idf_list.append(sims_list[i][1])

In [ ]:
#Create recipe dict- to be used in creating dataframe in next step - used to decode recipe id
recipe_dict = {k: v for k, v in enumerate(df_basic.RECIPE_NM_KO)}
recipe_dict

In [ ]:
#Use cosine similarity results to get the top 10 similar recipes for every recipe.
tf_idf_top  = []
similar_recipes_df = pd.DataFrame([])
same_item = []

#Get only top 11 largest values from the tf_idf_list - 1 recipe will be the same as itself (hence 12)
for i,item in enumerate(tf_idf_list):
    tf_idf_top.append(heapq.nlargest(11,enumerate(item), key=lambda x: x[1]))

#Remove the recipe value with 1.0 similarity - since it is the same recipe
for i,list_item in enumerate(tf_idf_top):
    for j,k in enumerate(list_item):
        if tf_idf_top[i][j][1] != 1.0:
            similar_recipes_df = similar_recipes_df.append(pd.DataFrame({'Similar_Recipe_ID': recipe_dict.get(tf_idf_top[i][j][0]),'TF-IDF Value': tf_idf_top[i][j][1],'Recipe_ID': recipe_dict.get(i)}, index=[0]), ignore_index=True)

In [ ]:
similar_recipes_df.to_csv('similar_recipes_top_10_tf_idf.csv',encoding='utf-8')

In [ ]:
similar_recipes_df = pd.read_csv('similar_recipes_top_10_tf_idf.csv')
similar_recipes_df = similar_recipes_df[similar_recipes_df['Recipe_ID'] != similar_recipes_df['Similar_Recipe_ID']]
print (similar_recipes_df.count())

In [ ]:
similar_recipes_df

In [ ]:
similar_recipes_df.to_csv('similar_recipes_tf_idf.csv',encoding='utf-8')

In [ ]:
similar_recipes_df

In [ ]:
#Create cosine similarity matrix for all recipes 27637*27637
#Since this is a huge matrix, the top 10 similar recipe logic is a better option.

names = [i for i in range(1,len(tf_idf_list))]
final_df = pd.DataFrame.from_dict(zip(names,tf_idf_list))

In [ ]:
final_df.head()

In [ ]:
df_basic['RECIPE_ID'] = df_basic.index+1
recipe_name_df = df_basic[['RECIPE_ID','RECIPE_NM_KO']]
final_df['RECIPE_ID'] = final_df.index+1

recipe_tf_idf_df = final_df.merge(recipe_name_df,how='left', left_on='RECIPE_ID', right_on='RECIPE_ID')

In [ ]:
recipe_tf_idf_df

In [ ]:
#Create a list from tfidf results
#This will be used to identify ingredient importance within every recipe

corpus_list = []
for doc in corpus_tfidf:
    corpus_list.append(doc)

In [ ]:
corpus_list[0]

In [ ]:
#Create a flat list to eliminate repetition of ingredients and create a dict to hold the results

flat_list = []
for sublist in ingr_list:
    for item in sublist:
        if item not in flat_list:
            flat_list.append(item)
ing_dict =  {k: v for k, v in enumerate(flat_list)}

In [ ]:
ing_dict

In [ ]:
dictionary.get(corpus_list[0][0][0])

In [ ]:
#Create a dataframe with tf-idf values per ingredient for every recipe.
corpus_df = pd.DataFrame([])

for i,list_item in enumerate(corpus_list):
    for j,k in enumerate(list_item):
        corpus_df = corpus_df.append(pd.DataFrame({'Ingredient': dictionary.get(corpus_list[i][j][0]),'TF-IDF Value': corpus_list[i][j][1],'Recipe_ID': i}, index=[0]), ignore_index=True)

In [ ]:
corpus_df.to_csv('ingredient_tf_idf.csv')

In [ ]:
df_basic['RECIPE_ID'] = df_basic.index
recipe_tf_idf_df = corpus_df.merge(df_basic,how='left', left_on='Recipe_ID', right_on='RECIPE_ID')
recipe_tf_idf_df = recipe_tf_idf_df[['Recipe_ID','RECIPE_NM_KO','Ingredient','TF-IDF Value']]

In [ ]:
recipe_tf_idf_df.to_csv('ingredient_recipe_tf_idf.csv',encoding='utf-8')

In [ ]:
for i, j in enumerate(recipe_tf_idf_df["RECIPE_NM_KO"].drop_duplicates()):
    if i == 30:
        break
    test = recipe_tf_idf_df[recipe_tf_idf_df["RECIPE_NM_KO"] == j]
    print(test.sort_values(by=['TF-IDF Value'], axis=0, ascending=False))